In [ ]:
import requests
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
from rdflib.namespace import FOAF, XSD
import re

def fetch_query(query):
    url = 'https://query.wikidata.org/sparql'
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, params={'query': query, 'format': 'json'})
    data = response.json()
    return data

def assert_iso8601_zulu(string):
    pattern = r"\d{4}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])T(0\d|1\d|2[0-3]):[0-5]\d:[0-5]\dZ"
    return re.match(pattern, string)


In [ ]:
WD = Namespace("http://www.wikidata.org/entity/")
SCHEMA = Namespace("http://schema.org/")
EX = Namespace("http://example.org/")
g = Graph()
g.bind("wd", WD)
g.bind("schema", SCHEMA)
g.bind("foaf", FOAF)
g.bind("ex", EX)
# load from football_data.ttl
g.parse("football_data.ttl", format="turtle")

## Players

In [ ]:
player_query = """
SELECT ?player ?playerLabel ?nationality ?nationalityLabel ?position ?positionLabel WHERE {
    ?player wdt:P106 wd:Q937857; # Instance of soccer player
            wdt:P27 ?nationality; # Nationality
            wdt:P21 wd:Q6581097;
            wdt:P413 ?position.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
player_uris = set()
for i in range(0, 1):
    player_results = fetch_query(player_query+f"LIMIT {100} OFFSET {i*100}")
    
    for player in player_results["results"]["bindings"]:
        player_uri = URIRef(player["player"]['value'])
        player_uris.add(player["player"]['value'])
        player_name = Literal(player["playerLabel"]['value'])
        nationality = Literal(player["nationalityLabel"]['value'])
        nationality_uri = URIRef(player["nationality"]['value'])
        position_uri = URIRef(player["position"]['value'])
        position = Literal(player["positionLabel"]['value'])
        
        # Adding triples for each player
        g.add((player_uri, RDF.type, SCHEMA.Person))
        g.add((player_uri, SCHEMA.name, player_name))
        g.add((player_uri, SCHEMA.nationality, nationality_uri))
        # add position
        g.add((player_uri, EX.position, position_uri))
        g.add((position_uri, RDF.type, EX.PlayerPosition))
        g.add((position_uri, SCHEMA.name, position))
        
        g.add((nationality_uri, RDF.type, SCHEMA.Country))
        g.add((nationality_uri, SCHEMA.name, nationality))
        
    print(f"{i} - {len(g)} triples. {len(player_uris)} players.".ljust(50), end="\r")

In [ ]:
# remove player_uri if they are already in the graph
player_uris = player_uris - set([str(player) for player in g.subjects(RDF.type, SCHEMA.Person)])

print('Number of triples:', len(g), '\nNumber of players:', len(player_uris))

## Teams

In [ ]:
# Assuming `player_uris` is a list of player URIs and `fetch_query` is a function to execute SPARQL queries

team_query_template = """
SELECT ?team ?teamLabel ?startTime ?endTime WHERE {
  BIND(%s AS ?player)
  ?player p:P54 ?playerTeamStatement.
  ?playerTeamStatement ps:P54 ?team;
                    pq:P580 ?startTime;
OPTIONAL { ?playerTeamStatement pq:P582 ?endTime. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
team_uris = set()
for player_uri in player_uris:
    team_query = team_query_template % f"<{player_uri}>"
    team_results = fetch_query(team_query)
    # Process team_results and add data to the knowledge graph
    for result in team_results["results"]["bindings"]:
        team_uri = URIRef(result["team"]["value"])
        team_uris.add(result["team"]["value"])
        team_name = Literal(result["teamLabel"]["value"])
        start_time = Literal(result["startTime"]["value"], datatype=XSD.date)
        end_time = result.get("endTime", {}).get("value")  # Handle potential missing end time
        player_ref = URIRef(player_uri)
        
        # Adding team information
        g.add((team_uri, RDF.type, SCHEMA.SportsTeam))
        g.add((team_uri, SCHEMA.name, team_name))
        
        g.add((player_ref, SCHEMA.memberOf, team_uri))
        
        # Create a unique identifier (BNode) for each player-team tenure
        tenure = BNode()
        g.add((tenure, RDF.type, SCHEMA.Role))
        g.add((tenure, SCHEMA.memberOf, team_uri))
        g.add((tenure, SCHEMA.startDate, start_time))
        if end_time and assert_iso8601_zulu(end_time):  # If there's an end date
            end_time = Literal(end_time, datatype=XSD.date)
            g.add((tenure, SCHEMA.endDate, end_time))
        player_ref = URIRef(player_uri)
        g.add((player_ref, SCHEMA.hasOccupation, tenure))
    print(f"{len(g)} triples. {len(team_uris)} teams.".ljust(50), end="\r")

In [ ]:
# remove teams that are already in the graph
team_uris = team_uris - set([str(team) for team in g.subjects(RDF.type, SCHEMA.SportsTeam)])

print('Number of triples:', len(g), '\nNumber of players:', len(player_uris), '\nNumber of teams:', len(team_uris))

#### Team details

In [ ]:
team_detail_query_template = """
SELECT ?teamLabel ?country ?league ?leagueLabel WHERE {
  BIND(<TEAM_URI> AS ?team)
  ?team wdt:P17 ?country; # Country the team is based in
        wdt:P118 ?league. # League the team competes in
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

def execute_team_detail_query(team_uri):
    # Prepare the query by replacing <TEAM_URI> with the current team's URI
    query = team_detail_query_template.replace("<TEAM_URI>", f"<{team_uri}>")
    # Execute the query and return the results
    return fetch_query(query)

# TODO ALSO SAVE LEAGUE URIS TO GET COUNTRY?
for team_uri in team_uris:
    # Execute query for the current team
    team_detail_results = execute_team_detail_query(team_uri)
    
    # Process results and add to the RDF graph
    for result in team_detail_results["results"]["bindings"]:
        # Assuming you have a function to convert the URI to a RDFLib URIRef if needed
        team_uri_ref = URIRef(team_uri)
        country_uri = URIRef(result["country"]["value"])
        league_uri = URIRef(result["league"]["value"])
        league = Literal(result["leagueLabel"]["value"])
        
        # Add country and league to the graph as instances of their respective types
        g.add((country_uri, RDF.type, SCHEMA.Country))
        g.add((league_uri, RDF.type, SCHEMA.SportsOrganization))
        g.add((league_uri, SCHEMA.name, league))
        
        # Link team to country and league
        g.add((team_uri_ref, SCHEMA.location, country_uri))
        g.add((team_uri_ref, SCHEMA.partOf, league_uri))
    print(f"{len(g)} triples.".ljust(20), end="\r")

In [ ]:
print('Number of triples:', len(g), '\nNumber of players:', len(player_uris), '\nNumber of teams:', len(team_uris))

#### Team victories

In [ ]:
# for each team_uri, get all sports seasons won (P2522)
team_victory_query_template = """
SELECT ?team ?tournament ?tournamentLabel ?organizer ?organizerLabel ?league ?leagueLabel ?startTime ?endTime WHERE {
  BIND(<TEAM_URI> AS ?team)
  ?team wdt:P2522 ?tournament.
  ?tournament wdt:P31 wd:Q27020041; # Instance of sports season
              wdt:P664 ?organizer;
              wdt:P3450 ?league.
                
    OPTIONAL {
        ?tournament wdt:P580 ?startTime;
                    wdt:P582 ?endTime.
    }
    OPTIONAL {
        ?tournament wdt:P585 ?startTime;
    }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

def execute_team_victory_query(team_uri):
    # Prepare the query by inserting the current team URI
    query = team_victory_query_template.replace("<TEAM_URI>", f"<{team_uri}>")
    # Execute the query
    return fetch_query(query)

# Initialize a set to store all unique tournament URIs encountered
tournament_uris = set()

# Loop through each team URI to execute the victory query
for team_uri in team_uris:
    # Execute the query for the current team
    team_victory_results = execute_team_victory_query(team_uri)
    
    # Process the results for each team's victories
    for result in team_victory_results["results"]["bindings"]:
        tournament_uri = URIRef(result["tournament"]["value"])
        tournament_uris.add(result["tournament"]["value"])  # Collect tournament URIs
        
        # Remaining processing, including adding data to the RDF graph
        tournament_label = Literal(result["tournamentLabel"]["value"])
        organizer_uri = URIRef(result["organizer"]["value"])
        organizer_label = Literal(result["organizerLabel"]["value"])
        league_uri = URIRef(result["league"]["value"])
        league_label = Literal(result["leagueLabel"]["value"])
        start_time = result.get("startTime", {}).get("value")
        end_time = result.get("endTime", {}).get("value")
        
        g.add((tournament_uri, RDF.type, SCHEMA.SportsSeason))
        g.add((tournament_uri, SCHEMA.name, tournament_label))
        g.add((URIRef(team_uri), SCHEMA.award, tournament_uri))  # Linking the victory to the team
        g.add((tournament_uri, SCHEMA.partOf, league_uri))
        g.add((league_uri, RDF.type, SCHEMA.SportsOrganization))
        g.add((league_uri, SCHEMA.name, league_label))
        
        g.add((tournament_uri, SCHEMA.organizer, organizer_uri))
        g.add((organizer_uri, RDF.type, SCHEMA.Organization))
        g.add((organizer_uri, SCHEMA.name, organizer_label))
        
        if start_time:
            start_time = Literal(start_time, datatype=XSD.date)
            g.add((tournament_uri, SCHEMA.startDate, start_time))
        if end_time:
            end_time = Literal(end_time, datatype=XSD.date)
            g.add((tournament_uri, SCHEMA.endDate, end_time))
    print(f"{len(g)} triples. {len(tournament_uris)} tournaments".ljust(20), end="\r")

In [ ]:
# remove tournament_uris that are already in the graph
tournament_uris = tournament_uris - set([str(tournament) for tournament in g.subjects(RDF.type, SCHEMA.SportsSeason)])

print('Number of triples:', len(g), '\nNumber of players:', len(player_uris), '\nNumber of teams:', len(team_uris), '\nNumber of tournaments:', len(tournament_uris))

## Tournaments

In [ ]:
# for each tournament_uri, get a name, number of participants (P1132), organizer (P664), winner (P1346)
tournament_details_query_template = """
SELECT ?tournamentLabel ?country ?numParticipants ?organizer ?organizerLabel ?winner ?winnerLabel WHERE {
  BIND(<TOURNAMENT_URI> AS ?tournament)
  ?tournament wdt:P17 ?country; # Country the tournament took place in
              wdt:P1132 ?numParticipants;
              wdt:P664 ?organizer;
              wdt:P1346 ?winner.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
# TODO CAN GET A LOT MORE INFORMATION: ranking, number of matches, number of points, number of wins, etc...
participating_teams_query_template = """
SELECT ?participatingTeam ?participatingTeamLabel WHERE {
  BIND(<TOURNAMENT_URI> AS ?tournament)
  ?tournament wdt:P1923 ?participatingTeam.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

def execute_query_for_tournament(tournament_uri, query_template):
    # Replace <TOURNAMENT_URI> with the current tournament URI in the query template
    query = query_template.replace("<TOURNAMENT_URI>", f"<{tournament_uri}>")
    # Execute the query
    return fetch_query(query)

# Loop through each tournament URI and execute both queries
for tournament_uri in tournament_uris:
    # Assuming tournaments_uri contains string representations of URIs
    tournament_details_results = execute_query_for_tournament(tournament_uri, tournament_details_query_template)
    participating_teams_results = execute_query_for_tournament(tournament_uri, participating_teams_query_template)
    
    tournament_uri = URIRef(tournament_uri)
    for result in tournament_details_results["results"]["bindings"]:
        tournament_name = Literal(result["tournamentLabel"]["value"])
        num_participants = Literal(result["numParticipants"]["value"], datatype=XSD.integer)
        
        organizer = URIRef(result["organizer"]["value"])
        organizer_name = Literal(result["organizerLabel"]["value"])
        
        winner = URIRef(result["winner"]["value"])
        winner_name = Literal(result["winnerLabel"]["value"])
        
        country = URIRef(result["country"]["value"])
        
        g.add((tournament_uri, SCHEMA.location, country))
        
        g.add((tournament_uri, RDF.type, SCHEMA.SportsEvent))
        g.add((tournament_uri, SCHEMA.name, tournament_name))
        g.add((tournament_uri, SCHEMA.numParticipants, num_participants))

        g.add((tournament_uri, SCHEMA.organizer, organizer))
        g.add((organizer, RDF.type, SCHEMA.Organization))
        g.add((organizer, SCHEMA.name, organizer_name))

        g.add((tournament_uri, SCHEMA.winner, winner))
        g.add((winner, RDF.type, SCHEMA.SportsTeam))
        g.add((winner, SCHEMA.name, winner_name))
    
    for result in participating_teams_results["results"]["bindings"]:
        if "participatingTeam" not in result:
            continue
        team_uri = URIRef(result["participatingTeam"]["value"])
        team_name = Literal(result["participatingTeamLabel"]["value"])
        
        g.add((team_uri, RDF.type, SCHEMA.SportsTeam))
        g.add((team_uri, SCHEMA.name, team_name))
        g.add((tournament_uri, SCHEMA.participant, team_uri))
    print(f"{len(g)} triples.".ljust(20), end="\r")

In [ ]:
print('Number of triples:', len(g), '\nNumber of players:', len(player_uris), '\nNumber of teams:', len(team_uris), '\nNumber of tournaments:', len(tournament_uris))

## Matches

In [ ]:
match_query = """
SELECT ?match ?matchLabel ?organizer ?location ?date ?winner ?tournament WHERE{
BIND(%s AS ?player)
?match wdt:P710 ?player;
       wdt:P664 ?organizer;
       wdt:P17 ?location;
       wdt:P585 ?date;
       wdt:P1346 ?winner;
       wdt:P361|wdt:P179 ?tournament.
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""


def execute_match_query(player_uri):
    query = match_query % f"<{player_uri}>"
    return fetch_query(query)

match_uris = set()

for player_uri in player_uris:
    match_results = execute_match_query(player_uri)
    
    for match in match_results["results"]["bindings"]:
        match_uri = URIRef(match["match"]["value"])
        match_uris.add(match["match"]["value"])
        match_name = Literal(match["matchLabel"]["value"])
        organizer_uri = URIRef(match["organizer"]["value"])
        location_uri = URIRef(match["location"]["value"])
        date = Literal(match["date"]["value"], datatype=XSD.date)
        winner_uri = URIRef(match["winner"]["value"])
        tournament_uri = URIRef(match["tournament"]["value"])
        
        g.add((match_uri, RDF.type, SCHEMA.SportsEvent))
        g.add((match_uri, SCHEMA.name, match_name))
        g.add((match_uri, SCHEMA.organizer, organizer_uri))
        g.add((match_uri, SCHEMA.location, location_uri))
        g.add((match_uri, SCHEMA.startDate, date))
        g.add((match_uri, SCHEMA.winner, winner_uri))
        g.add((match_uri, SCHEMA.partOf, tournament_uri))
        
        g.add((organizer_uri, RDF.type, SCHEMA.Organization))
        g.add((location_uri, RDF.type, SCHEMA.Country))
        g.add((winner_uri, RDF.type, SCHEMA.SportsTeam))
        g.add((tournament_uri, RDF.type, SCHEMA.SportsSeason))
    print(f"{len(g)} triples. {len(match_uris)} matches.".ljust(20), end="\r")

In [ ]:
# remove matches already in graph
match_uris = match_uris - set([str(match) for match in g.subjects(RDF.type, SCHEMA.SportsEvent)])

print('Number of triples:', len(g), '\nNumber of players:', len(player_uris), '\nNumber of teams:', len(team_uris), '\nNumber of tournaments:', len(tournament_uris), '\nNumber of matches:', len(match_uris))

In [ ]:
goal_query = """
SELECT ?player (COUNT(?player) AS ?goals) WHERE{
BIND(%s AS ?match)
?match wdt:P1363 ?player. # point scored by
}
GROUP BY ?player
"""

def execute_goal_query(match_uri):
    query = goal_query % f"<{match_uri}>"
    return fetch_query(query)

for match_uri in match_uris:
    goal_results = execute_goal_query(match_uri)
    match_uri = URIRef(match_uri)
    for goal in goal_results["results"]["bindings"]:
        player_uri = URIRef(goal["player"]["value"])
        goals = Literal(goal["goals"]["value"], datatype=XSD.integer)
        # Add triples that say "player X scored Y goals in match Z"
        score = BNode()
        g.add((score, RDF.type, EX.Score))
        g.add((score, EX.player, player_uri))
        g.add((score, EX.goals, goals))
        g.add((score, EX.match, match_uri))
    print(f"{len(g)} triples.".ljust(20), end="\r")

In [ ]:
print('Number of triples:', len(g), '\nNumber of players:', len(player_uris), '\nNumber of teams:', len(team_uris), '\nNumber of tournaments:', len(tournament_uris), '\nNumber of matches:', len(match_uris))

In [ ]:
# Save the graph to a file
g.serialize("football_data.ttl", format="turtle")